## Problem 2:

#### (a)

In [1]:
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
import scipy.optimize as opt

data = np.loadtxt('MacroSeries.txt', delimiter=',')
c = data[:,0]
k = data[:,1]
w = data[:,2]
r = data[:,3]

In [23]:
def get_zseries(w, k, alpha):
    denom = (1 - alpha) * (k ** alpha)
    return np.log(w / denom)

def log_lik_z(w, k, alpha, rho, mu, sigma):
    z_array = get_zseries(w, k, alpha)
    zprev_arr = np.roll(z_array, 1)
    zprev_arr[0] = mu
    dist_mean = (rho * zprev_arr) + ((1 - rho) * mu)
    
    z_pdf = (1/(sigma * np.sqrt(2 * np.pi)) *
            np.exp( - (z_array - dist_mean)**2 / (2 * sigma**2)))
    
    z_pdf[np.isclose(z_pdf, np.zeros_like(z_pdf), atol=1e-25)] = 1e-10
    
    ln_pdf_vals = np.log(z_pdf, where=(z_pdf>0))
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def crit(params, *args):
    alpha, rho, mu, sigma = params
    w, k = args
    log_lik_val = log_lik_z(w, k, alpha, rho, mu, sigma)
    return -log_lik_val

alpha_init = 0.457
rho_init = 0.72
mu_init = 9.52
sigma_init = 1
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
args_mle = (w, k)

results = opt.minimize(crit, params_init, args=(args_mle), method='L-BFGS-B',
                       bounds=((1e-10, 1-1e-10),(1e-10-1, 1-1e-10),
                               (1e-10, None), (1e-10, None)))

alpha_mle, rho_mle, mu_mle, sigma_mle = results.x
log_lik_mle = log_lik_z(w, k, alpha_mle, rho_mle, mu_mle, sigma_mle)

print("alpha_mle=", alpha_mle, "\nrho_mle=", rho_mle,
      "\nmu_mle=", mu_mle, "\nsigma_mle=", sigma_mle,
      "\nlog likelihood=", log_lik_mle)


print("\nInverse Hessian:\n", results.hess_inv.todense())

alpha_mle= 0.457533527217 
rho_mle= 0.720477219933 
mu_mle= 9.52247950197 
sigma_mle= 0.0919961708086 
log likelihood= 96.7069080835

Inverse Hessian:
 [[  2.33433680e+02  -1.52242979e+02  -3.32325513e+03  -4.06037969e+00]
 [ -1.52242979e+02   9.97695949e+01   2.16653271e+03   2.60496409e+00]
 [ -3.32325513e+03   2.16653271e+03   4.73127715e+04   5.78855810e+01]
 [ -4.06037969e+00   2.60496409e+00   5.78855810e+01   7.47849224e-02]]


#### (b)

In [45]:
def get_zseriesb(r, k, alpha):
    denom = alpha * (k ** (alpha - 1))
    return np.log(r / denom)


def log_lik_zb(r, k, alpha, rho, mu, sigma):
    z_array = get_zseriesb(r, k, alpha)
    zprev_arr = np.roll(z_array, 1)
    zprev_arr[0] = mu
    dist_mean = (rho * zprev_arr) + ((1 - rho) * mu)
    z_pdf = (1/(sigma * np.sqrt(2 * np.pi)) *
            np.exp( - (z_array - dist_mean)**2 / (2 * sigma**2)))
    z_pdf[np.isclose(z_pdf, np.zeros_like(z_pdf), atol=1e-25)] = 1e-10
    
    ln_pdf_vals = np.log(z_pdf, where=(z_pdf>0))
    log_lik_val = ln_pdf_vals.sum()
    
    return log_lik_val

def crit(params, *args):
    alpha, rho, mu, sigma = params
    r, k = args
    log_lik_val = log_lik_zb(r, k, alpha, rho, mu, sigma)
    return -log_lik_val

alpha_init = 0.5
rho_init = 0.9
mu_init = 5
sigma_init = 1
params_init = np.array([alpha_init, rho_init, mu_init, sigma_init])
args_mle = (r, k)

results = opt.minimize(crit, params_init, args=(args_mle), method='L-BFGS-B',
                       bounds=((1e-10, 1-1e-10),(1e-10-1, 1-1e-10),
                               (1e-10, None), (1e-10, None)))

alpha_mleb, rho_mleb, mu_mleb, sigma_mleb = results.x
log_lik_mleb = log_lik_zb(r, k, alpha_mleb, rho_mleb, mu_mleb, sigma_mleb)

print("alpha_mle=", alpha_mleb, "\nrho_mle=", rho_mleb,
      "\nmu_mle=", mu_mleb, "\nsigma_mle=", sigma_mleb,
      "\nlog likelihood=", log_lik_mleb)
print("\nInverse Hessian:\n", results.hess_inv.todense())

alpha_mle= 0.457480492396 
rho_mle= 0.720514858938 
mu_mle= 9.37093661628 
sigma_mle= 0.0919962400639 
log likelihood= 96.7069080265

Inverse Hessian:
 [[  4.11978560e+01  -1.99083683e+01  -7.42493161e+02  -3.86831408e-01]
 [ -1.99083683e+01   1.10011852e+01   3.58973481e+02   1.79011091e-01]
 [ -7.42493161e+02   3.58973481e+02   1.33816921e+04   6.97072710e+00]
 [ -3.86831408e-01   1.79011091e-01   6.97072710e+00   3.72245129e-03]]


#### (c)

In [51]:
import scipy

def z_pdf(z_array, mu, sigma, rho):
    dist_mean = (rho * 10) + ((1 - rho) * mu)
    z_pdf = (1/(sigma * np.sqrt(2 * np.pi)) *
            np.exp( - (z_array - dist_mean)**2 / (2 * sigma**2)))
    return z_pdf


pdf_z = lambda z: z_pdf(z, mu_mle, sigma_mle, rho_mle)

zprime = np.log(1 / (alpha_mle * (7500000 ** (alpha_mle - 1))))
result = 1 - scipy.integrate.quad(pdf_z, 0, zprime)[0]
print("Probability of interest rates > 1:", result)

Probability of interest rates > 1: 0.9999995960078569
